In [1]:
import numpy as np
import pickle
from utils import plot_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
y_train = pickle.load( open( "y_train.pickle", "rb" ) )
y_test = pickle.load( open( "y_test.pickle", "rb" ) )
depth_prob_train = pickle.load( open( "ytrain_proba.pickle", "rb" ) )
depth_prob_test = pickle.load( open( "yprob.pickle", "rb" ) )
skeleton_prob_train = pickle.load( open( "skeleton_pred_train.pickle", "rb" ) )
skeleton_prob_test = pickle.load( open( "skeleton_pred_test.pickle", "rb" ) )
inertia_prob_train = pickle.load( open( "inertia_pred_train.pickle", "rb" ) )
inertia_prob_test = pickle.load( open( "inertia_pred_test.pickle", "rb" ) )

In [3]:
skeleton_prob_train.shape, skeleton_prob_test.shape, depth_prob_train.shape, depth_prob_test.shape, inertia_prob_train.shape, inertia_prob_test.shape

((604, 27), (260, 27), (604, 27), (260, 27), (604, 27), (260, 27))

### Soft Voting: Predicts the class label based on the maximum of the averages of predicted probabilities (all sensors)

In [4]:
ensemble_prob_train = (depth_prob_train + skeleton_prob_train + inertia_prob_train)/3
ensemble_pred_train = ensemble_prob_train.argmax(axis=1)+1
ensemble_prob_test = (depth_prob_test + skeleton_prob_test + inertia_prob_test)/3
ensemble_pred_test = ensemble_prob_test.argmax(axis=1)+1

In [5]:
# Test Set Accuracy
print("Soft Voting Ensemble:")
accuracy = accuracy_score(y_train.argmax(axis=1), ensemble_pred_train)
print("Train Accuracy: %.2f%%" % (accuracy*100))
accuracy = accuracy_score(y_test.argmax(axis=1), ensemble_pred_test)
print("Test Accuracy: %.2f%%" % (accuracy*100))

Soft Voting Ensemble:
Train Accuracy: 100.00%
Test Accuracy: 92.31%


### Soft Voting: Predicts the class label based on the maximum of the averages of predicted probabilities (Skeleton and Inertia)

In [6]:
ensemble_prob_train = (skeleton_prob_train + inertia_prob_train)/2
ensemble_pred_train = ensemble_prob_train.argmax(axis=1)+1
ensemble_prob_test = (skeleton_prob_test + inertia_prob_test)/2
ensemble_pred_test = ensemble_prob_test.argmax(axis=1)+1

In [7]:
# Test Set Accuracy
print("Soft Voting Ensemble:")
accuracy = accuracy_score(y_train.argmax(axis=1), ensemble_pred_train)
print("Train Accuracy: %.2f%%" % (accuracy*100))
accuracy = accuracy_score(y_test.argmax(axis=1), ensemble_pred_test)
print("Test Accuracy: %.2f%%" % (accuracy*100))

Soft Voting Ensemble:
Train Accuracy: 100.00%
Test Accuracy: 94.62%


### Stacked Ensemble (all sensors) using Logistic Regression

In [8]:
trainX = np.concatenate((skeleton_prob_train,depth_prob_train,inertia_prob_train),axis=1)
testX = np.concatenate((skeleton_prob_test,depth_prob_test,inertia_prob_test),axis=1)

In [9]:
trainY = y_train.argmax(axis=1)

In [10]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10, solver='lbfgs', multi_class='auto', random_state=123)
logreg.fit(trainX, trainY)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=123, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
# Train Set Accuracy
print("Stacking Ensemble:")
accuracy = accuracy_score(y_train.argmax(axis=1), logreg.predict(trainX))
print("Train Accuracy: %.2f%%" % (accuracy*100))
accuracy = accuracy_score(y_test.argmax(axis=1), logreg.predict(testX))
print("Test Accuracy: %.2f%%" % (accuracy*100))

Stacking Ensemble:
Train Accuracy: 100.00%
Test Accuracy: 91.15%


### Stacked Ensemble (Skeleton and Inertia sensors) using Logistic Regression

In [12]:
trainX = np.concatenate((skeleton_prob_train,inertia_prob_train),axis=1)
testX = np.concatenate((skeleton_prob_test,inertia_prob_test),axis=1)

In [13]:
trainY = y_train.argmax(axis=1)

In [14]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=10, solver='lbfgs', multi_class='auto', random_state=123)
logreg.fit(trainX, trainY)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=123, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
# Train Set Accuracy
print("Stacking Ensemble:")
accuracy = accuracy_score(y_train.argmax(axis=1), logreg.predict(trainX))
print("Train Accuracy: %.2f%%" % (accuracy*100))
accuracy = accuracy_score(y_test.argmax(axis=1), logreg.predict(testX))
print("Test Accuracy: %.2f%%" % (accuracy*100))

Stacking Ensemble:
Train Accuracy: 100.00%
Test Accuracy: 94.62%
